## 예스24티켓 크롤링
#### 공연 개별 상세페이지에서 정보 가져오기

In [ ]:
from selenium import webdriver
import time

path="C:/Users/YYJ/Desktop/myPython/chromedriver.exe"
driver=webdriver.Chrome(path)

In [ ]:
def get_data(ptype):
    
    p_keys=['Type', 'Title', 'Subtitle', 'Venue', 'open_Date', 'end_Date', 'Performer']
    subtitle=venue=o_date=e_date=''
    performer=[]
    
    # Title
    title=driver.find_element_by_class_name('rn-big-title').text
    
    # Date
    o_date=driver.find_element_by_class_name('ps-date').text.split(' ', 2)[0]
    e_date=driver.find_element_by_class_name('ps-date').text.split(' ', 2)[2]
    
    # Venue
    try:
        venue=driver.find_element_by_class_name('ps-location').text
    except:
        venue=driver.find_element_by_class_name('ps-location2').text
    
    # Performer
    for perf in driver.find_elements_by_class_name('rn-product-peole'):
        performer.append(perf.text)
        
    p_value=[ptype, title, subtitle, venue, o_date, e_date, performer]
    data=dict(zip(p_keys, p_value))
    print(data)
    
    return data

In [ ]:
def get_link():
    
    link=[]
    
    # 무한 스크롤
    last_height=driver.execute_script("return document.body.scrollHeight;")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(0.5)
        # Calculate new scroll height and compare with last scroll height
        new_height=driver.execute_script("return document.body.scrollHeight")
        if new_height==last_height:
            break
        last_height=new_height
    
    # 상세페이지로 접근하는 웹요소를 찾아 url에 입력할 번호만 파싱
    for row in driver.find_elements_by_class_name('ms-list-imgs'):
        for col in row.find_elements_by_tag_name('a'):
            s=col.get_attribute('onclick')
            s1=s.split('(')[1]
            s2=s1.split(')')[0]
            link.append(s2)
    
    return link

In [ ]:
p_type={'15456':'콘서트', '15457':'뮤지컬', '15458':'연극', '15459':'클래식', '15460':'전시'}
p_data=[]       # 공연정보 리스트

for key in p_type:
    
    # [장르 - 전체보기 - 신상품순] 페이지 접근
    driver.get("http://ticket.yes24.com/New/Genre/GenreList.aspx?genretype=1&genre="+key)
    time.sleep(0.5)
    driver.find_element_by_xpath('/html/body/section/p[1]/span[2]/a').click()
    
    # 각 공연의 상세페이지로 접근하기 위해 클릭할 웹요소 p_link에 가져오기
    p_link=get_link()
    
    # p_link의 주소로 상세페이지에 접근하여 데이터 가져오기
    for link in p_link:
        
        driver.get('http://ticket.yes24.com/Perf/'+link)
        time.sleep(0.5)
            
        # 상세페이지에서 얻은 공연정보 데이터를 p_data에 추가하기
        p_data.append(get_data(p_type[key]))
        